# Baseline model for batch monitoring example

In [458]:
import requests
import datetime
import pandas as pd

from evidently import ColumnMapping
from evidently.report import Report
from evidently.metrics import ColumnDriftMetric, DatasetDriftMetric, DatasetMissingValuesMetric

from joblib import load, dump
from tqdm import tqdm

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

In [ ]:
import os

files = [('green_tripdata_2024-03.parquet', './data'),('green_tripdata_2022-02.parquet', './data'), ('green_tripdata_2022-01.parquet', './data')]

print("Download files:")
for file, path in files:
    save_path = f"{path}/{file}"
    if os.path.exists(save_path):
        print(f"{file} already exists")
    else:
        url = f"https://d37ci6vzurychx.cloudfront.net/trip-data/{file}"
        resp = requests.get(url, stream=True)
        with open(save_path, "wb") as handle:
            for data in tqdm(resp.iter_content(),
                             desc=f"{file}",
                             postfix=f"save to {save_path}",
                             total=int(resp.headers["Content-Length"])):
                handle.write(data)

In [460]:
current_data = pd.read_parquet('data/green_tripdata_2024-03.parquet')

In [ ]:
current_data.describe()

In [ ]:
current_data.shape

In [463]:
# create target
current_data["duration_min"] = current_data.lpep_dropoff_datetime - current_data.lpep_pickup_datetime
current_data.duration_min = current_data.duration_min.apply(lambda td : float(td.total_seconds())/60)

In [464]:
# filter out outliers
current_data = current_data[(current_data.duration_min >= 0) & (current_data.duration_min <= 60)]
current_data = current_data[(current_data.passenger_count > 0) & (current_data.passenger_count <= 8)]

In [ ]:
current_data.duration_min.hist()

In [466]:
# data labeling
target = "duration_min"
num_features = ["passenger_count", "trip_distance", "fare_amount", "total_amount"]
cat_features = ["PULocationID", "DOLocationID"]

In [ ]:
current_data.shape

In [468]:
train_data = current_data[:30000]
val_data = current_data[30000:]

In [469]:
model = LinearRegression()

In [ ]:
model.fit(train_data[num_features + cat_features], train_data[target])

In [471]:
train_preds = model.predict(train_data[num_features + cat_features])
train_data['prediction'] = train_preds

In [472]:
val_preds = model.predict(val_data[num_features + cat_features])
val_data['prediction'] = val_preds

In [ ]:
print(mean_absolute_error(train_data.duration_min, train_data.prediction))
print(mean_absolute_error(val_data.duration_min, val_data.prediction))

# Dump model and reference data

In [474]:
with open('models/lin_reg.bin', 'wb') as f_out:
    dump(model, f_out)

In [475]:
val_data.to_parquet('data/reference.parquet')

# Evidently Report

In [476]:
column_mapping = ColumnMapping(
    target=None,
    prediction='prediction',
    numerical_features=num_features,
    categorical_features=cat_features
)

In [477]:
report = Report(metrics=[
    ColumnDriftMetric(column_name='prediction'),
    DatasetDriftMetric(),
    DatasetMissingValuesMetric()
]
)

In [478]:
report.run(reference_data=train_data, current_data=val_data, column_mapping=column_mapping)

In [ ]:
report.show(mode='inline')

In [480]:
result = report.as_dict()

In [ ]:
result

In [ ]:
#prediction drift
result['metrics'][0]['result']['drift_score']

In [ ]:
#number of drifted columns
result['metrics'][1]['result']['number_of_drifted_columns']

In [ ]:
#share of missing values
result['metrics'][2]['result']['current']['share_of_missing_values']

# Evidently Dashboard

In [485]:
from evidently.metric_preset import DataDriftPreset, DataQualityPreset

from evidently.ui.workspace import Workspace
from evidently.ui.dashboards import DashboardPanelCounter, DashboardPanelPlot, CounterAgg, PanelValue, PlotType, ReportFilter
from evidently.renderers.html_widgets import WidgetSize

In [486]:
ws = Workspace("workspace")

In [ ]:
project = ws.create_project("NYC Taxi Data Quality Project")
project.description = "My project descriotion"
project.save()

In [ ]:
import pandas as pd

# Assuming 'val_data' is the validation data
val_data['lpep_pickup_datetime'] = pd.to_datetime(val_data['lpep_pickup_datetime'])

# Filter the data for March 2024
march_2024_data = val_data.loc[val_data['lpep_pickup_datetime'].between('2024-03-01', '2024-03-31')]

# Calculate the median (quantile = 0.5) for fare_amount in March 2024
median_fare_march_2024 = march_2024_data['fare_amount'].quantile(0.5)

print(f"The median (quantile = 0.5) for fare_amount in March 2024 is: {median_fare_march_2024}")


In [ ]:
regular_report = Report(
    metrics=[
        DataQualityPreset()
    ],
    timestamp=datetime.datetime(2024,3,28)
)

regular_report.run(reference_data=None,
                  current_data=val_data.loc[val_data.lpep_pickup_datetime.between('2024-03-28', '2024-03-29', inclusive="left")],
                  column_mapping=column_mapping)

regular_report

In [503]:
ws.add_report(project.id, regular_report)

In [ ]:
#configure the dashboard
project.dashboard.add_panel(
    DashboardPanelCounter(
        filter=ReportFilter(metadata_values={}, tag_values=[]),
        agg=CounterAgg.NONE,
        title="NYC taxi data dashboard"
    )
)

project.dashboard.add_panel(
    DashboardPanelPlot(
        filter=ReportFilter(metadata_values={}, tag_values=[]),
        title="Inference Count",
        values=[
            PanelValue(
                metric_id="DatasetSummaryMetric",
                field_path="current.number_of_rows",
                legend="count"
            ),
        ],
        plot_type=PlotType.BAR,
        size=WidgetSize.HALF,
    ),
)

project.dashboard.add_panel(
    DashboardPanelPlot(
        filter=ReportFilter(metadata_values={}, tag_values=[]),
        title="Number of Missing Values",
        values=[
            PanelValue(
                metric_id="DatasetSummaryMetric",
                field_path="current.number_of_missing_values",
                legend="count"
            ),
        ],
        plot_type=PlotType.LINE,
        size=WidgetSize.HALF,
    ),
)

project.save()

In [ ]:
regular_report = Report(
    metrics=[
        DataQualityPreset()
    ],
    timestamp=datetime.datetime(2042,3,29)
)

regular_report.run(reference_data=None,
                  current_data=val_data.loc[val_data.lpep_pickup_datetime.between('2024-03-29', '2024-03-30', inclusive="left")],
                  column_mapping=column_mapping)

regular_report

In [506]:
ws.add_report(project.id, regular_report)